In [1]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# check to see that the gpu is available
print(tf..device_lib.list_local_devices())

In [2]:
import modules.midiMethods as midiMethods
reload(midiMethods)
from modules.midiMethods import *

import modules.dataMethods as dataMethods
reload(dataMethods)
from modules.dataMethods import *

import modules.models as models
reload(models)

import modules.mlClasses as mlClasses
reload(mlClasses)


<module 'modules.mlClasses' from '/home/ubuntu/storage/380-music-representation/modules/mlClasses.py'>

## Performance Representation Sequence Generation
This purpose of this notebook is to generate sequences using the Performance Representation/oore representation.
Weights are loaded into a new model with the same architecture as at training time, but with sequence length of 1, and hidden states retained across batches. The choice of temperature is vitally important on the quality of the new sequences. Low temperatures emphasize more likely events; higher temperatures 'even out' the softmax distribution.
See here for a very good tutorial on text generation: https://www.tensorflow.org/tutorials/text/text_generation

### Model Without Chroma

In [24]:
# tf.train.latest_checkpoint('weights\first_attempt')
hidden_state = 512
lstm_layers = 3
# n_events should be 242 or 333, depending on which version of oore was used
prediction_model = models.create_ooremodel(batch_size=1, stateful=True, hidden_state_size=hidden_state, lstm_layers=lstm_layers, n_events=242, chroma=False)

Model: "3layerLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(1, 600, 1)]             0         
_________________________________________________________________
lambda_11 (Lambda)           (1, 600, 242)             0         
_________________________________________________________________
lstm_30 (LSTM)               (1, 600, 512)             1546240   
_________________________________________________________________
lstm_31 (LSTM)               (1, 600, 512)             2099200   
_________________________________________________________________
lstm_32 (LSTM)               (1, 600, 512)             2099200   
_________________________________________________________________
dense_10 (Dense)             (1, 600, 242)             124146    
Total params: 5,868,786
Trainable params: 5,868,786
Non-trainable params: 0
______________________________________________

In [25]:
# load some weights
# if, at this point, there is an error in which two shapes differ by 12, that means the model should be rebuilt with/without chroma
prediction_model.load_weights('models/oore/oore8/32-1.92.hdf5')
# prediction_model.load_weights('weights/note_bin/model2_60epochs512state3layer.h5')
# choose an input sequence to start things off with
start = 'spa4'
input_events = pm2oore2(pretty_midi.PrettyMIDI('midi/starts/spa2.mid'))

In [ ]:
for temperature in np.linspace(0,1,11):
    new_seq_length = 600
    new_seq = models.generate_ooremusic(prediction_model, new_seq_length, temperature=temperature, input_events=input_events)
    new_seq_list = [int(event) for event in new_seq]
    # convert to prettymidi object and write to file
    pm = oore2pm2(new_seq_list)
    pm.write(f'midi/oore/{start}-{new_seq_length}long-{temperature}.mid')

### Model With Chroma
Code for generating sequences with chroma using PF does exist in models.py, but is currently untested.